In [1]:
# Import PuLP modeler functions
from pulp import *

# Creates a list of all the supply nodes
Warehouses = ["A", "B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
          "B": 4000,
          "C": 100}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700,}

# Creates a list of costs of each transportation path
costs = [   #Bars
         #1 2 3 4 5
         [2,4,5,2,1],#A   Warehouses
         [3,1,3,2,3], #B
         [0,0,0,0,0]
         ]

In [2]:
# The cost data is made into a dictionary
costs = makeDict([Warehouses,Bars],costs,0)

In [10]:
costs

defaultdict(<function pulp.pulp.__makeDict.<locals>.<lambda>()>,
            {'A': defaultdict(<function pulp.pulp.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}),
             'B': defaultdict(<function pulp.pulp.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3}),
             'C': defaultdict(<function pulp.pulp.__makeDict.<locals>.<lambda>()>,
                         {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0})})

In [3]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer Distribution Problem",LpMinimize)

In [4]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]

In [11]:
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5'),
 ('C', '1'),
 ('C', '2'),
 ('C', '3'),
 ('C', '4'),
 ('C', '5')]

In [12]:
(Warehouses,Bars)

(['A', 'B', 'C'], ['1', '2', '3', '4', '5'])

In [5]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)

In [13]:
vars

{'A': {'1': Route_A_1,
  '2': Route_A_2,
  '3': Route_A_3,
  '4': Route_A_4,
  '5': Route_A_5},
 'B': {'1': Route_B_1,
  '2': Route_B_2,
  '3': Route_B_3,
  '4': Route_B_4,
  '5': Route_B_5},
 'C': {'1': Route_C_1,
  '2': Route_C_2,
  '3': Route_C_3,
  '4': Route_C_4,
  '5': Route_C_5}}

In [17]:
for (i,j) in Routes:
    print(vars[i][j])

Route_A_1
Route_A_2
Route_A_3
Route_A_4
Route_A_5
Route_B_1
Route_B_2
Route_B_3
Route_B_4
Route_B_5
Route_C_1
Route_C_2
Route_C_3
Route_C_4
Route_C_5


In [6]:
# The objective function is added to 'prob' first
prob += lpSum([vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

In [7]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([vars[w][b] for b in Bars])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += lpSum([vars[w][b] for w in Warehouses])>=demand[b], "Sum_of_Products_into_Bar%s"%b

In [8]:
# The problem data is written to an .lp file
prob.writeLP("BeerDistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1700.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_C_1 = 0.0
Route_C_2 = 0.0
Route_C_3 = 100.0
Route_C_4 = 0.0
Route_C_5 = 0.0


In [9]:
# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Total Cost of Transportation =  8300.0
